### Trouver les évolutions des lésions en comparant deux timepoints différents d'un seul patient (non automatisé)

In [ ]:
import SimpleITK as sitk
import math
import csv
from pathlib import Path

# MODIFIABLE PARAMETERS : Modifier selon le patient et les timepoints souhaités
PATIENT_ID = "P1"   # Patient_id varie de P1 à P53 mais pas pour de P54 à P75 car il n'y a qu'un seul timepoint
DEPARTURE_TIMEPOINT = "T1"
ARRIVAL_TIMEPOINT = "T2"

data_path = Path.cwd().parent.parent / "data"

variation_folder_path = data_path / "11_lesion_evolution_results"
variation_folder_path.mkdir(parents = True, exist_ok = True)

departure_mask_path = data_path / f"01_MSLesSeg_Dataset/train/{PATIENT_ID}/{DEPARTURE_TIMEPOINT}/{PATIENT_ID}_{DEPARTURE_TIMEPOINT}_MASK.nii.gz"
arrival_mask_path = data_path / f"01_MSLesSeg_Dataset/train/{PATIENT_ID}/{ARRIVAL_TIMEPOINT}/{PATIENT_ID}_{ARRIVAL_TIMEPOINT}_MASK.nii.gz"

if not departure_mask_path.exists():
    arg = "Chemin non validé (attention de ne pas mettre un patient de P54 à P75 car il n'y a qu'un seul timepoint T1) "
    raise ValueError(arg)

if sum(1 for sub_dir in departure_mask_path.parent.iterdir() if sub_dir.is_dir()) < 2:
    arg = f"Il y a un seul timepoint T1 pour le patient {PATIENT_ID}, changer de patient svp."
    raise ValueError(arg)


In [ ]:
#TODO